## Model Comparison

| Parameter | Constant |
|---|---|
| Input modality | Grayscale image |
| Input resolution | **256 × 256** |
| Input channels | 1 |
| Output layer | Linear |
| Optimization algorithm | SGD |


| Component | Linear Model (Baseline) | CNN Baseline | CNN + Anti-Feature-Collapse |
|---------|-------------------------|--------------|-----------------------------|
| Feature extraction | None (raw pixels) | Convolutional layers | Convolutional layers |
| Convolution layers | – | 3× Conv + ReLU + MaxPool | 3× Conv + **BatchNorm** + ReLU + MaxPool |
| Feature vector size | 65 536 | 65 536 (64×32×32) | 65 536 (64×32×32) |
| Hidden fully-connected layers | None | None | None |
| Non-linearity before output | None | ReLU (in convs) | ReLU (in convs) |
| Anti-collapse mechanisms | None | None | BatchNorm / Dropout / Feature decorrelation |
| Loss function | Cross-entropy | Cross-entropy | Cross-entropy + regularization |
| Feature collapse analysis | Baseline reference | Intermediate | Primary focus |


## Reusable code

### Prepare venv

In [ ]:
dir = "../../dataset/variance_big_256/"
labels = ["canter", "trot", "walk"]

rng = 42

batch_size = 8
epochs = 10

#### imports

In [ ]:
import numpy as np
import os
import cv2

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader

#### load variances

In [ ]:
files = sorted(os.listdir(dir))

X_list, y_list = [], []

for label in labels:
    for file in sorted(os.listdir(dir + label)):
        var = np.load(dir + label + "/" + file).astype(np.float32)
        X_list.append(var)
        y_list.append(labels.index(label))

X = np.stack(X_list, axis=0)       # shape: (N, 256, 256)
y = np.array(y_list)               # shape: (N,)

#### train/test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rng)